<a href="https://colab.research.google.com/github/Austfi/SNOWPACKforPatrollers/blob/feature/snodas_nc/Build_SNODAS_netCDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# SNODAS netCDF Dataset Builder (Colorado)

This notebook downloads SNODAS (Snow Data Assimilation System) data and builds reusable netCDF files organized by year, subset to Colorado boundaries. These files can be referenced by all other notebooks for fast point queries without re-downloading data.

**Features:**
- Downloads SNODAS data from NSIDC
- Extracts ALL available variables from tar files (snow depth, SWE, accumulation, melt, etc.)
- Subsets to Colorado mountain region (37-41°N, -109 to -104°W)
- Builds one netCDF file per year (e.g., `snodas_co_2024.nc`)
- Incremental updates: only downloads missing dates
- Handles grid configuration changes (pre/post Oct 2013)
- Compressed storage for efficient access


In [ ]:
# @title Install and Import Required Libraries
%pip install xarray netcdf4 pandas numpy

import os
import struct
import tarfile
import gzip
import urllib.request
import urllib.error
import re
from io import BytesIO
from datetime import datetime, timedelta
from typing import Optional, Tuple

import numpy as np
import pandas as pd
import xarray as xr

print("✓ Libraries imported successfully")


Note: you may need to restart the kernel to use updated packages.
✓ Libraries imported successfully


In [ ]:
# @title Configuration Parameters

# @markdown ### Date Range
start_year = 2020  # @param {type:"integer"}
end_year = 2025  # @param {type:"integer"}

# @markdown ### Directories
output_directory = "snodas_nc"  # @param {type:"string"}
cache_directory = "snodas_cache"  # @param {type:"string"}

# @markdown ### Options
subset_to_colorado = True  # @param {type:"boolean"}
rebuild_existing = False  # @param {type:"boolean"}

print("Configuration:")
print(f"  Years: {start_year} to {end_year}")
print(f"  Output directory: {output_directory}")
print(f"  Cache directory: {cache_directory}")
print(f"  Subset to Colorado: {subset_to_colorado}")
print(f"  Rebuild existing: {rebuild_existing}")


Configuration:
  Years: 2024 to 2024
  Output directory: snodas_nc
  Cache directory: snodas_cache
  Rebuild existing: False


In [ ]:
# Core Functions for SNODAS Data Processing

SNODAS_NODATA = -9999

# Colorado bounds (mountain region, excluding plains)
CO_BOUNDS = {
    'lat_min': 37.0,
    'lat_max': 41.0,
    'lon_min': -109.0,  # West boundary
    'lon_max': -104.0   # East boundary (excluding plains)
}

# SNODAS product code to variable name mapping
# Common SNODAS variables found in tar files
VAR_NAMES = {
    '1036': 'snow_depth',        # Snow Depth (mm -> m)
    '1034': 'swe',               # Snow Water Equivalent (mm -> m)
    '1038': 'snow_accumulation', # Snow Depth Accumulation (mm -> m)
    '1039': 'snow_melt',         # Snow Melt (mm -> m)
    '1033': 'snow_cover',        # Snow Cover (percentage)
    '1037': 'snow_depth_change', # Snow Depth Change (mm -> m)
}

# Grid configurations (detected from file size)
GRID_CONFIGS = {
    'old': {
        'XMIN': -124.73375000000000,
        'YMAX': 52.87458333333333,
        'XMAX': -66.94208333333333,
        'YMIN': 24.94958333333333,
        'NCOLS': 6935,
        'NROWS': 3351,
        'name': 'Pre-Oct-2013'
    },
    'new': {
        'XMIN': -124.73333333333333,
        'YMAX': 52.87500000000000,
        'XMAX': -66.94166666666667,
        'YMIN': 24.95000000000000,
        'NCOLS': 3353,
        'NROWS': 3353,
        'name': 'Post-Oct-2013'
    }
}


def subset_to_colorado(lat_array, lon_array, data_array):
    """
    Subset SNODAS grid to Colorado mountain region.
    
    Args:
        lat_array: 1D latitude array
        lon_array: 1D longitude array
        data_array: 2D data array (NROWS x NCOLS) or 3D (time, NROWS, NCOLS)
    
    Returns:
        Tuple of (lat_subset, lon_subset, data_subset)
    """
    # Find indices within Colorado bounds
    lat_mask = (lat_array >= CO_BOUNDS['lat_min']) & (lat_array <= CO_BOUNDS['lat_max'])
    lon_mask = (lon_array >= CO_BOUNDS['lon_min']) & (lon_array <= CO_BOUNDS['lon_max'])
    
    # Get indices
    lat_indices = np.where(lat_mask)[0]
    lon_indices = np.where(lon_mask)[0]
    
    # Subset coordinate arrays
    lat_subset = lat_array[lat_indices]
    lon_subset = lon_array[lon_indices]
    
    # Subset data (rows = lat, cols = lon)
    if data_array.ndim == 2:
        # 2D array: (NROWS, NCOLS)
        data_subset = data_array[np.ix_(lat_indices, lon_indices)]
    elif data_array.ndim == 3:
        # 3D array: (time, NROWS, NCOLS)
        # First subset rows (latitude), then columns (longitude)
        data_subset = data_array[:, lat_indices, :][:, :, lon_indices]
    else:
        raise ValueError(f"Unsupported array dimension: {data_array.ndim}")
    
    return lat_subset, lon_subset, data_subset


def get_snodas_grid(date_str: str, cache_dir: str, subset_co: bool = True, verbose: bool = False) -> Optional[Tuple[dict, dict]]:
    """
    Extract all available SNODAS variables for a specific date.
    
    Args:
        date_str: Date string in YYYYMMDD format
        cache_dir: Directory to cache tar files
        subset_co: If True, subset to Colorado bounds
        verbose: Print progress messages
    
    Returns:
        Tuple of (variables_dict, grid_info) or None if failed
        - variables_dict: Dictionary with variable names as keys and 2D arrays as values (in meters)
        - grid_info: Dictionary with grid metadata and coordinate arrays
    """
    # Construct URL
    tar_filename = f"SNODAS_{date_str}.tar"
    data_base = "https://noaadata.apps.nsidc.org/NOAA/G02158/masked"
    year = date_str[:4]
    month = date_str[4:6]
    month_names = ["01_Jan", "02_Feb", "03_Mar", "04_Apr", "05_May", "06_Jun",
                   "07_Jul", "08_Aug", "09_Sep", "10_Oct", "11_Nov", "12_Dec"]
    month_dir = month_names[int(month) - 1]
    data_url = f"{data_base}/{year}/{month_dir}/{tar_filename}"
    
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, tar_filename)
    
    try:
        # Download or use cache
        if os.path.exists(cache_path):
            if verbose:
                print(f"    Using cached: {date_str}")
            with open(cache_path, 'rb') as f:
                tar_data = BytesIO(f.read())
        else:
            if verbose:
                print(f"    Downloading: {date_str}")
            with urllib.request.urlopen(data_url, timeout=60) as response:
                tar_data = BytesIO(response.read())
                with open(cache_path, 'wb') as f:
                    f.write(tar_data.getvalue())
            tar_data.seek(0)
        
        # Extract all available variables from tar file
        variables_dict = {}
        grid_config = None
        grid_info = None
        
        with tarfile.open(fileobj=tar_data, mode='r') as tar:
            # Get all .dat.gz files in the tar
            for member in tar.getmembers():
                if member.name.endswith('.dat.gz'):
                    # Extract product code from filename (e.g., us_ssmv11036tS...1036...)
                    # Look for 4-digit codes in the filename
                    codes = re.findall(r'(\d{4})', member.name)
                    
                    # Find matching product code
                    var_code = None
                    for code in codes:
                        if code in VAR_NAMES:
                            var_code = code
                            break
                    
                    if var_code is None:
                        # Try to extract from common patterns
                        if '1036' in member.name:
                            var_code = '1036'
                        elif '1034' in member.name:
                            var_code = '1034'
                        elif '1038' in member.name:
                            var_code = '1038'
                        elif '1039' in member.name:
                            var_code = '1039'
                        elif '1033' in member.name:
                            var_code = '1033'
                        elif '1037' in member.name:
                            var_code = '1037'
                        else:
                            continue  # Skip unknown variables
                    
                    var_name = VAR_NAMES.get(var_code, f'var_{var_code}')
                    
                    # Extract and decompress
                    var_file = tar.extractfile(member)
                    with gzip.open(var_file, 'rb') as gz_file:
                        data = gz_file.read()
                    
                    # Detect grid from file size (only need to do once)
                    if grid_config is None:
                        num_values = len(data) // 2
                        for config in GRID_CONFIGS.values():
                            if num_values == config['NCOLS'] * config['NROWS']:
                                grid_config = config
                                break
                        
                        if grid_config is None:
                            if verbose:
                                print(f"    Error: Unknown grid size for {date_str}")
                            return None
                        
                        # Create coordinate arrays (only once)
                        XMIN = grid_config['XMIN']
                        YMAX = grid_config['YMAX']
                        XMAX = grid_config['XMAX']
                        YMIN = grid_config['YMIN']
                        NCOLS = grid_config['NCOLS']
                        NROWS = grid_config['NROWS']
                        CELLSIZE_X = (XMAX - XMIN) / NCOLS
                        CELLSIZE_Y = (YMAX - YMIN) / NROWS
                        
                        lon = np.linspace(XMIN + CELLSIZE_X/2, XMAX - CELLSIZE_X/2, NCOLS)
                        lat = np.linspace(YMAX - CELLSIZE_Y/2, YMIN + CELLSIZE_Y/2, NROWS)
                    
                    # Parse binary data
                    NCOLS = grid_config['NCOLS']
                    NROWS = grid_config['NROWS']
                    values = struct.unpack(f">{NCOLS * NROWS}h", data)
                    var_array = np.array(values, dtype=np.float32).reshape((NROWS, NCOLS))
                    
                    # Convert to meters and handle nodata
                    # SNODAS stores values in mm, convert to m
                    var_array = var_array.astype(np.float32) / 1000.0
                    var_array[var_array < 0.01] = 0.0
                    var_array[var_array == SNODAS_NODATA / 1000.0] = np.nan
                    
                    # Subset to Colorado if requested
                    if subset_co:
                        lat_sub, lon_sub, var_array = subset_to_colorado(lat, lon, var_array)
                        if grid_info is None:
                            # Store subset coordinates
                            grid_info = {
                                'lat': lat_sub,
                                'lon': lon_sub,
                                'config': grid_config,
                                'date': date_str,
                                'subset_co': True
                            }
                    else:
                        if grid_info is None:
                            grid_info = {
                                'lat': lat,
                                'lon': lon,
                                'config': grid_config,
                                'date': date_str,
                                'subset_co': False
                            }
                    
                    variables_dict[var_name] = var_array
        
        if not variables_dict:
            if verbose:
                print(f"    Error: No variables found in {date_str}")
            return None
        
        return variables_dict, grid_info
        
    except Exception as e:
        if verbose:
            print(f"    Error processing {date_str}: {e}")
        import traceback
        if verbose:
            traceback.print_exc()
        return None


def build_year_dataset(year: int, output_dir: str, cache_dir: str, 
                      subset_co: bool = True, rebuild: bool = False, 
                      verbose: bool = True) -> bool:
    """
    Build or update netCDF dataset for a single year with all available variables.
    
    Args:
        year: Year to process (e.g., 2024)
        output_dir: Directory to save netCDF files
        cache_dir: Directory for cached tar files
        subset_co: If True, subset to Colorado bounds
        rebuild: If True, rebuild even if file exists
        verbose: Print progress messages
    
    Returns:
        True if successful, False otherwise
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Update filename based on subset
    if subset_co:
        output_file = os.path.join(output_dir, f"snodas_co_{year}.nc")
    else:
        output_file = os.path.join(output_dir, f"snodas_{year}.nc")
    
    # Generate all dates for the year
    start_date = pd.Timestamp(f"{year}-01-01")
    # For current year, only go up to today
    if year == pd.Timestamp.now().year:
        end_date = pd.Timestamp.now().date()
    else:
        end_date = pd.Timestamp(f"{year}-12-31").date()
    
    all_dates = pd.date_range(start_date, end_date, freq='D')
    
    if verbose:
        print(f"\nProcessing year {year} ({len(all_dates)} days)...")
        if subset_co:
            print(f"  Subsetting to Colorado: {CO_BOUNDS['lat_min']}-{CO_BOUNDS['lat_max']}°N, {CO_BOUNDS['lon_min']}-{CO_BOUNDS['lon_max']}°W")
    
    # Check if file exists and get existing dates
    existing_dates = set()
    if os.path.exists(output_file) and not rebuild:
        try:
            ds_existing = xr.open_dataset(output_file)
            existing_dates = set(pd.to_datetime(ds_existing.time.values).strftime('%Y%m%d'))
            ds_existing.close()
            if verbose:
                print(f"  Found existing file with {len(existing_dates)} dates")
        except Exception as e:
            if verbose:
                print(f"  Warning: Could not read existing file: {e}")
    
    # Determine which dates to process
    dates_to_process = []
    for date in all_dates:
        date_str = date.strftime('%Y%m%d')
        if date_str not in existing_dates:
            dates_to_process.append(date_str)
    
    if not dates_to_process:
        if verbose:
            print(f"  ✓ Year {year} is complete. No new dates to process.")
        return True
    
    if verbose:
        print(f"  Processing {len(dates_to_process)} new/missing dates...")
    
    # Collect grids for all variables
    all_variables = {}  # {var_name: [grid1, grid2, ...]}
    dates = []
    grid_info = None
    failed_dates = []
    
    for idx, date_str in enumerate(dates_to_process, 1):
        if verbose and idx % 30 == 0:
            print(f"    Progress: {idx}/{len(dates_to_process)} ({100*idx/len(dates_to_process):.1f}%)")
        
        result = get_snodas_grid(date_str, cache_dir, subset_co=subset_co, verbose=False)
        if result is not None:
            var_dict, info = result
            dates.append(pd.to_datetime(date_str, format='%Y%m%d'))
            
            # Store grids for each variable
            for var_name, grid in var_dict.items():
                if var_name not in all_variables:
                    all_variables[var_name] = []
                all_variables[var_name].append(grid)
            
            if grid_info is None:
                grid_info = info
        else:
            failed_dates.append(date_str)
    
    if not all_variables:
        if verbose:
            print(f"  ✗ No data retrieved for year {year}")
        return False
    
    if verbose:
        print(f"  ✓ Retrieved {len(dates)}/{len(dates_to_process)} dates")
        print(f"  Variables found: {', '.join(sorted(all_variables.keys()))}")
        if failed_dates:
            print(f"  ⚠ Failed dates: {len(failed_dates)}")
    
    # Stack into 3D arrays for each variable
    lat = grid_info['lat']
    lon = grid_info['lon']
    
    # Create data_vars dictionary for xarray Dataset
    data_vars = {}
    for var_name, grids in all_variables.items():
        var_3d = np.stack(grids, axis=0)
        data_vars[var_name] = (['time', 'latitude', 'longitude'], var_3d)
    
    # Create xarray Dataset with all variables
    ds_new = xr.Dataset(
        data_vars,
        coords={
            'time': pd.to_datetime(dates),
            'latitude': lat,
            'longitude': lon,
        },
        attrs={
            'title': f'SNODAS Dataset - {year}' + (' (Colorado)' if subset_co else ''),
            'source': 'NOAA NSIDC SNODAS',
            'units': 'meters',
            'nodata': 'NaN',
            'grid_config': grid_info['config']['name'],
            'variables': ', '.join(sorted(all_variables.keys())),
            'colorado_bounds': str(CO_BOUNDS) if subset_co else 'full_conus'
        }
    )
    
    # Merge with existing data if updating
    if os.path.exists(output_file) and not rebuild and existing_dates:
        try:
            ds_existing = xr.open_dataset(output_file)
            # Combine datasets, ensuring time dimension is sorted
            ds_combined = xr.concat([ds_existing, ds_new], dim='time')
            ds_combined = ds_combined.sortby('time')
            ds_existing.close()
            ds_new = ds_combined
            if verbose:
                print(f"  ✓ Merged with existing data")
        except Exception as e:
            if verbose:
                print(f"  Warning: Could not merge with existing file: {e}")
            # Continue with new data only
    
    # Save to netCDF with compression for all variables
    encoding = {}
    for var_name in all_variables.keys():
        encoding[var_name] = {
            'zlib': True,
            'complevel': 4,
            'dtype': 'float32'
        }
    
    if verbose:
        print(f"  Saving to {output_file}...")
    
    ds_new.to_netcdf(output_file, encoding=encoding)
    
    if verbose:
        file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
        print(f"  ✓ Saved: {file_size_mb:.2f} MB")
        print(f"    Dimensions: {ds_new.dims}")
        print(f"    Variables: {', '.join(sorted(all_variables.keys()))}")
        print(f"    Time range: {ds_new.time.min().values} to {ds_new.time.max().values}")
    
    return True


print("✓ Core functions loaded")


✓ Core functions loaded


In [ ]:
# @title Build SNODAS netCDF Files

print("=" * 60)
print("SNODAS netCDF Dataset Builder")
print("=" * 60)

# Validate year range
if start_year > end_year:
    raise ValueError(f"start_year ({start_year}) must be <= end_year ({end_year})")

current_year = pd.Timestamp.now().year
if end_year > current_year:
    print(f"⚠ Warning: end_year ({end_year}) is in the future. Clamping to {current_year}.")
    end_year = current_year

# Create output directory
os.makedirs(output_directory, exist_ok=True)
os.makedirs(cache_directory, exist_ok=True)

print(f"\nProcessing years {start_year} to {end_year}")
print(f"Output directory: {output_directory}")
print(f"Cache directory: {cache_directory}")
print(f"Rebuild existing: {rebuild_existing}")
print("\n" + "=" * 60)

# Process each year
successful_years = []
failed_years = []

for year in range(start_year, end_year + 1):
    try:
        success = build_year_dataset(
            year=year,
            output_dir=output_directory,
            cache_dir=cache_directory,
            subset_co=subset_to_colorado,
            rebuild=rebuild_existing,
            verbose=True
        )
        if success:
            successful_years.append(year)
        else:
            failed_years.append(year)
    except Exception as e:
        print(f"\n✗ Error processing year {year}: {e}")
        failed_years.append(year)
        import traceback
        traceback.print_exc()

# Summary
print("\n" + "=" * 60)
print("Summary")
print("=" * 60)
print(f"✓ Successful years: {len(successful_years)}")
if successful_years:
    print(f"  {successful_years}")
if failed_years:
    print(f"✗ Failed years: {len(failed_years)}")
    print(f"  {failed_years}")
print("\n" + "=" * 60)


SNODAS netCDF Dataset Builder

Processing years 2024 to 2024
Output directory: snodas_nc
Cache directory: snodas_cache
Rebuild existing: False


Processing year 2024 (366 days)...
  Processing 366 new/missing dates...
    Progress: 30/366 (8.2%)
    Progress: 60/366 (16.4%)
    Progress: 90/366 (24.6%)
    Progress: 120/366 (32.8%)
    Progress: 150/366 (41.0%)
    Progress: 180/366 (49.2%)
    Progress: 210/366 (57.4%)
    Progress: 240/366 (65.6%)
    Progress: 270/366 (73.8%)
    Progress: 300/366 (82.0%)
    Progress: 330/366 (90.2%)
    Progress: 360/366 (98.4%)
  ✓ Retrieved 366/366 dates
  Saving to snodas_nc/snodas_2024.nc...
  ✓ Saved: 1478.74 MB
    Dimensions: FrozenMappingWarningOnValuesAccess({'time': 366, 'latitude': 3351, 'longitude': 6935})
    Time range: 2024-01-01T00:00:00.000000000 to 2024-12-31T00:00:00.000000000

Summary
✓ Successful years: 1
  [2024]



In [ ]:
# @title Verify Created Files and Example Usage

import glob

print("=" * 60)
print("Created Files")
print("=" * 60)

# List all netCDF files (both full and Colorado subset)
nc_files = sorted(glob.glob(os.path.join(output_directory, "snodas*.nc")))

if not nc_files:
    print("No netCDF files found.")
else:
    print(f"Found {len(nc_files)} netCDF file(s):\n")
    
    total_size = 0
    for nc_file in nc_files:
        file_size = os.path.getsize(nc_file)
        file_size_mb = file_size / (1024 * 1024)
        total_size += file_size
        filename = os.path.basename(nc_file)
        
        # Try to get dataset info
        try:
            ds = xr.open_dataset(nc_file)
            time_range = f"{ds.time.min().values} to {ds.time.max().values}"
            num_dates = len(ds.time)
            dims_info = ds.dims
            ds.close()
            print(f"  {filename}")
            print(f"    Size: {file_size_mb:.2f} MB")
            print(f"    Dates: {num_dates} ({time_range})")
            print(f"    Dimensions: {dims_info}")
            print()
        except Exception as e:
            print(f"  {filename}")
            print(f"    Size: {file_size_mb:.2f} MB")
            print(f"    Error reading: {e}")
            print()
    
    total_size_gb = total_size / (1024 * 1024 * 1024)
    print(f"Total size: {total_size_gb:.2f} GB")

print("\n" + "=" * 60)
print("Example Usage in Other Notebooks")
print("=" * 60)
print("""
# Load a specific year's dataset (Colorado subset)
import xarray as xr
ds = xr.open_dataset('snodas_nc/snodas_co_2024.nc')

# Query a point value for a specific date
latitude = 39.5261  # Example: Vail Pass, Colorado
longitude = -106.2131
date_str = '2024-01-15'

# Access snow depth
point_depth = ds.sel(
    time=date_str,
    latitude=latitude,
    longitude=longitude,
    method='nearest'
).snow_depth.values

# Access SWE (if available)
if 'swe' in ds.data_vars:
    point_swe = ds.sel(
        time=date_str,
        latitude=latitude,
        longitude=longitude,
        method='nearest'
    ).swe.values
    print(f"SWE on {date_str}: {point_swe:.3f} m")

print(f"Snow depth on {date_str}: {point_depth:.3f} m")

# Get time series for a point (all variables)
point_data = ds.sel(
    latitude=latitude,
    longitude=longitude,
    method='nearest'
)

# Convert to pandas DataFrame for easy plotting
df = point_data.to_dataframe()

# Close dataset when done
ds.close()
""")


Created Files
Found 1 netCDF file(s):

  snodas_2024.nc
    Size: 1478.74 MB
    Dates: 366 (2024-01-01T00:00:00.000000000 to 2024-12-31T00:00:00.000000000)
    Dimensions: FrozenMappingWarningOnValuesAccess({'time': 366, 'latitude': 3351, 'longitude': 6935})

Total size: 1.44 GB

Example Usage in Other Notebooks

# Load a specific year's dataset
import xarray as xr
ds = xr.open_dataset('snodas_nc/snodas_2024.nc')

# Query a point value for a specific date
latitude = 39.5261  # Example: Vail Pass, Colorado
longitude = -106.2131
date_str = '2024-01-15'

point_value = ds.sel(
    time=date_str,
    latitude=latitude,
    longitude=longitude,
    method='nearest'
).snow_depth.values

print(f"Snow depth on {date_str}: {point_value:.3f} m")

# Get time series for a point
point_series = ds.sel(
    latitude=latitude,
    longitude=longitude,
    method='nearest'
).snow_depth

# Convert to pandas Series for easy plotting
snow_depth_ts = point_series.to_pandas()

# Close dataset when done
ds.c

: 